In [130]:
import pandas as pd
from datasets import load_dataset
from sklearn.model_selection import train_test_split

# ¿Que dicen los LLM de nuestros datos?

Primero importaremos los datos y le aplicaremos la función de rating

In [131]:
# leemos el csv
df = pd.read_csv("threads.csv")
df = df.drop(["source", "review_date"], axis=1)
# change the column names to match the HuggingFace dataset format
df = df.rename(columns={"review": "text", "rating": "label"})

# función para convertir el rating a palabras
def ratingTransform(rating):
    if rating <= 2:
        return 0
    elif rating <= 4:
        return 1
    else:
        return 2

# Transform the ratings to positive, neutral, and negative
df["label"] = df["label"].apply(ratingTransform)

train, test = train_test_split(df, test_size=0.33)

# guardamos los datos en un csv
train.to_csv("train.csv", index=False)
test.to_csv("test.csv", index=False)

# cargamos los datos en un dataset de huggingface
dataset = load_dataset("csv", data_files={"train": "train.csv", "test": "test.csv"})

Extracting data files: 100%|██████████| 2/2 [00:00<00:00, 154.26it/s]
Generating train split: 22049 examples [00:00, 485786.82 examples/s]
Generating test split: 10861 examples [00:00, 473488.57 examples/s]


## Usando Tokenizers

Los tokenizers son el proceso de convertir una secuencia de texto en una secuencia de tokens (números que hacen referencia a palabras).

In [132]:
# Entrenemos un modelo DistilBERT
from transformers import AutoTokenizer
tokenizer = AutoTokenizer.from_pretrained("distilbert-base-uncased")

#max_length = 512
# Tokenizamos los datos
def tokenize_data(example):
    return tokenizer(example['review_description'], padding='max_length', truncation=True)

# tokenizamos los datos
dataset = dataset.map(tokenize_data, batched=True)

Map: 100%|██████████| 10861/10861 [00:01<00:00, 5812.50 examples/s]


## Cargamos el modelo DistilBERT

DistilBERT es un modelo de BERT que es más pequeño y rápido, pero que mantiene una precisión similar a la de BERT.

In [133]:
from transformers import AutoModelForSequenceClassification, TrainingArguments

# Cargamos un modelo pre-entrenado
model = AutoModelForSequenceClassification.from_pretrained("distilbert-base-uncased", num_labels=3)

batch_size = 8
number_of_epochs = 7
logging_steps = len(dataset['train']) // batch_size
steps = (len(dataset['train']) // batch_size) * number_of_epochs
warmup_steps = int(0.2*steps)

training_args = TrainingArguments(
                                  num_train_epochs=number_of_epochs, 
                                  load_best_model_at_end=True,
                                  evaluation_strategy='steps', 
                                  save_strategy='steps',
                                  learning_rate=2e-5,
                                  logging_steps=logging_steps,
                                  warmup_steps= warmup_steps,
                                  save_steps=1000,
                                  eval_steps=500,
                                  output_dir="fine-tuned-distilbert-base-uncased"
                                  )

# shuffle the dataset
train_dataset = dataset['train'].shuffle(seed=10) 
eval_dataset = dataset['test'].shuffle(seed=10)

Some weights of DistilBertForSequenceClassification were not initialized from the model checkpoint at distilbert-base-uncased and are newly initialized: ['pre_classifier.weight', 'classifier.bias', 'classifier.weight', 'pre_classifier.bias']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


# Fine-tuning de DistilBERT

El fine-tuning es un proceso de entrenamiento en el que se usa un modelo pre-entrenado y se ajusta para que se adapte a los datos específicos del problema. En este caso, usaremos el modelo pre-entrenado de DistilBERT y lo ajustaremos para que se adapte a nuestros datos de entrenamiento.

In [134]:
from transformers import Trainer
import numpy as np
from datasets import load_metric

# Funcion para poder computar las metricas
def compute_metrics(eval_pred):
    logits, labels = eval_pred
    predictions = np.argmax(logits, axis=-1)

    # Cargar las metricas
    accuracy_metric = load_metric("accuracy")
    f1_metric = load_metric("f1")
    precision_metric = load_metric("precision")
    recall_metric = load_metric("recall")
    
    # Calcular las metricas
    accuracy  = accuracy_metric.compute(predictions=predictions, references=labels)["accuracy"]
    f1        = f1_metric.compute(predictions=predictions, references=labels, average="weighted")["f1"]
    precision = precision_metric.compute(predictions=predictions, references=labels, average="weighted")["precision"]
    recall    = recall_metric.compute(predictions=predictions, references=labels, average="weighted")["recall"]

    metrics = {
        'accuracy': accuracy,
        'f1': f1,
        'precision': precision,
        'recall': recall
    }

    return metrics

""" prev func
#Función para computar las metricas
def compute_metrics(eval_pred):
    # load the metrics to use
    load_accuracy = load_metric("accuracy")
    load_f1 = load_metric("f1")

    logits, labels = eval_pred
    predictions = np.argmax(logits, axis=-1)
    # calculate the mertic using the predicted and true value 
    accuracy = load_accuracy.compute(predictions=predictions, references=labels)
    f1 = load_f1.compute(predictions=predictions, references=labels, average="weighted")
    return {"accuracy": accuracy, "f1score": f1}
"""

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=eval_dataset,
    compute_metrics=compute_metrics,
)

trainer.train()

  3%|▎         | 500/19299 [01:41<1:03:34,  4.93it/s]

c:\Users\frgon\UChile\Mineria\App-Reviewer\venv2\Lib\site-packages\sklearn\metrics\_classification.py:1471: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))

  3%|▎         | 500/19299 [03:20<1:03:34,  4.93it/s]

{'eval_loss': 0.8851959705352783, 'eval_accuracy': 0.6289476107172451, 'eval_f1': 0.5639379312396521, 'eval_precision': 0.5150621330237951, 'eval_recall': 0.6289476107172451, 'eval_runtime': 98.789, 'eval_samples_per_second': 109.941, 'eval_steps_per_second': 13.746, 'epoch': 0.18}


                                                       
  5%|▌         | 1000/19299 [06:40<1:01:54,  4.93it/s]

{'eval_loss': 0.6858401894569397, 'eval_accuracy': 0.7304115643126784, 'eval_f1': 0.698458599270395, 'eval_precision': 0.7078432111473476, 'eval_recall': 0.7304115643126784, 'eval_runtime': 98.1154, 'eval_samples_per_second': 110.696, 'eval_steps_per_second': 13.841, 'epoch': 0.36}


                                                        
  8%|▊         | 1500/19299 [10:02<59:37,  4.97it/s]

...

100%|█████████▉| 19292/19299 [2:07:42<00:01,  4.93it/s]  

{'loss': 0.2545, 'learning_rate': 9.066770286898517e-09, 'epoch': 7.0}


100%|██████████| 19299/19299 [2:07:44<00:00,  2.52it/s]

{'train_runtime': 7664.0357, 'train_samples_per_second': 20.139, 'train_steps_per_second': 2.518, 'train_loss': 0.45131043809900334, 'epoch': 7.0}


TrainOutput(global_step=19299, training_loss=0.45131043809900334, metrics={'train_runtime': 7664.0357, 'train_samples_per_second': 20.139, 'train_steps_per_second': 2.518, 'train_loss': 0.45131043809900334, 'epoch': 7.0})

# Evaluación del modelo

Ahora que hemos entrenado nuestro modelo, podemos evaluarlo en el conjunto de datos de prueba, para ver cómo se desempeña en datos que nunca ha visto antes.

In [135]:
trainer.evaluate()

100%|██████████| 1358/1358 [01:42<00:00, 13.31it/s]


{'eval_loss': 0.638457715511322,
 'eval_accuracy': 0.7639259736672498,
 'eval_f1': 0.7514273856034948,
 'eval_precision': 0.7494562038253741,
 'eval_recall': 0.7639259736672498,
 'eval_runtime': 102.3634,
 'eval_samples_per_second': 106.102,
 'eval_steps_per_second': 13.266,
 'epoch': 7.0}

# Interpretación del modelo

Ahora veremos que palabras son las que más influyen en la predicción del modelo.

In [169]:
# Interpretabilidad del modelo
from transformers_interpret import SequenceClassificationExplainer

# load the explainer
explainer = SequenceClassificationExplainer(
    model,
    tokenizer
)

sample = "best app! I really like it. better than twitter"
explanation = explainer(sample)
print(f"\nsample: \"{sample}\" -> Debería ser Positivo (Label 2)")
print("Visualization: ")
explainer.visualize(sample)

sample = "its neither good nor bad."
explanation = explainer(sample)
print(f"\nsample: \"{sample}\" -> Debería ser Neutral (Label 1)")
print("Visualization: ")
explainer.visualize(sample)

sample = "I recently installed this app due to its popularity, but my initial impression is quite negative. I haven't found anything that I like about it."
explanation = explainer(sample)
print(f"\nsample: \"{sample}\" -> Debería ser Negativo (Label 0)")
print("Visualization: ")
explainer.visualize(sample)

""


sample: "best app! I really like it. better than twitter" -> Debería ser Positivo (Label 2)
Visualization: 


True Label,Predicted Label,Attribution Label,Attribution Score,Word Importance
2,LABEL_2 (0.87),LABEL_2,1.43,[CLS] best app ! i really like it . better than twitter [SEP]



sample: "its neither good nor bad." -> Debería ser Neutral (Label 1)
Visualization: 


True Label,Predicted Label,Attribution Label,Attribution Score,Word Importance
1,LABEL_1 (0.44),LABEL_1,0.87,[CLS] its neither good nor bad . [SEP]



sample: "I recently installed this app due to its popularity, but my initial impression is quite negative. I haven't found anything that I like about it." -> Debería ser Negativo (Label 0)
Visualization: 


''